## Загрузка и изучение файла

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
display(df)
df.info()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Открыла файл, в датасете 3214 строк. Есть информация о количестве звонков, суммарной длительности звонков в минутах, количестве sms-сообщений, израсходованный интернет-трафик в Мб и то, каким тарифом пользовался пользователь в течение месяца («Ультра» — 1, «Смарт» — 0).

## Разбивка данных на выборки

In [3]:
df_train, df_valid = train_test_split(df, test_size=0.4, random_state=12345)
df_valid, df_test = train_test_split(df_valid, test_size=0.5, random_state=12345)
print(df_train.shape)
print(df_valid.shape)
print(df_test.shape)
1928+643+643

(1928, 5)
(643, 5)
(643, 5)


3214

Разбила данные на обучающую, валидационную и тестовую выборки в соотношении 3:1:1. Получила размеры наборов для проверки корректности действий. 

## Исследование моделей

In [4]:
df_train_features = df_train.drop(['is_ultra'], axis=1)
df_train_target = df_train['is_ultra']
df_valid_features = df_valid.drop(['is_ultra'], axis=1)
df_valid_target = df_valid['is_ultra']
df_test_features = df_test.drop(['is_ultra'], axis=1)
df_test_target = df_test['is_ultra']

for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(df_train_features, df_train_target)
    predictions_valid = model.predict(df_valid_features) 
    print(accuracy_score(df_valid_target, predictions_valid))
max_depth = (0.7542768273716952, 0.7822706065318819, 0.7853810264385692, 0.7791601866251944, 0.7791601866251944)
max(max_depth)

0.7542768273716952
0.7822706065318819
0.7853810264385692
0.7791601866251944
0.7791601866251944


0.7853810264385692

Исследовала модель "дерево решений". Поменяла гиперпараметр глубины дерева от 1 до 5 в цикле. Качество модели при изм-ии глубины практически не отличается. 

In [5]:
best_model = None
best_result = 0
for est in range(1, 11):
    model = RandomForestClassifier(random_state=12345, n_estimators=est) 
    model.fit(df_train_features, df_train_target) 
    result = model.score(df_valid_features, df_valid_target) 
    if result > best_result:
        best_model = model
        best_result = result

print("Accuracy наилучшей модели на валидационной выборке:", best_result)
best_model

Accuracy наилучшей модели на валидационной выборке: 0.7853810264385692


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=12345,
                       verbose=0, warm_start=False)

Исследовала модель случайного леса с числом деревьев от 1 до 10. Качество наилучшей модели на валидационной выборке получилось 0.79. С учетом ожидания качества не ниже 0.75, 0.79 это довольно неплохой рез-т.  

In [6]:
model = LogisticRegression(solver='liblinear', random_state=12345)
model.fit(df_train_features, df_train_target) 
result = model.score(df_valid_features, df_valid_target) 
print("Accuracy модели логистической регрессии на валидационной выборке:", result)

Accuracy модели логистической регрессии на валидационной выборке: 0.7589424572317263


Исследовала логистическую регрессию. Гиперпараметры логистической регрессии настраивать не нужно. Качество модели на валидационных данных 0.76

## Проверка моделей на тестовой выборке

In [7]:
model = DecisionTreeClassifier(random_state=12345, max_depth=3)
model.fit(df_train_features, df_train_target)
predictions_valid = model.predict(df_test_features)
accuracy_score(df_test_target, predictions_valid)

0.7791601866251944

In [8]:
model = RandomForestClassifier(random_state=12345, n_estimators=10) 
model.fit(df_train_features, df_train_target) 
result = model.score(df_test_features, df_test_target) 
result

0.7807153965785381

In [9]:
model = LogisticRegression(solver='liblinear', random_state=12345)
model.fit(df_train_features, df_train_target) 
result = model.score(df_test_features, df_test_target) 
result

0.7402799377916018

Измерила качество моделей на тестовой выборке с лучшими подобранными параметрами: лучший результат показало дерево решений, затем идет случайный лес. Худший и не соответствующий пороговому значению 0.75 результат у логистической регрессии.

## 5. (бонус) Проверьте модели на адекватность

In [10]:
dummy_clf = DummyClassifier(strategy="most_frequent", random_state=0)
dummy_clf.fit(df_train_features, df_train_target)
dummy_clf.score(df_test_features, df_test_target)

0.6842923794712286

Проверила модель на адекватность при помощи классификатора DummyClassifier. На базе константной модели получила качество модели на уровне 0.68, что на порядок хуже качества, которое выдают тестируемые модели (<0.75). Можно сделать вывод, что качество тестируемых моделей с подобранными параметрами лучше константной модели, что говорит об адекватности тестируемых моделей.    